# Оценка для отзыва

Это будет задача классификации. Есть датасет `X`, состоящий из отзывов, и соответсвенный ему `y`, в котором значения бывают 0 или 1. И оценивать мы будем тоже бинарно -- хороший отзыв или плохой.

Данные кроулятся с сайта http://www.banki.ru. Изначально там пятибальная оценка, однако отзывов 2, 3 и 4 сильно меньше, чем остальных, поэтому было решено переделать это в бинарную оценку. Считаем, что все, что меньше или равно 2 -- это 0, а больше или равно 3 -- 1. Граница тоже выбрана не случайно, дело в том, что отрицательных отзывов больше, поэтому 3 было решено отнести к положительным оценкам. Обработка данных выполнена в файле `data_preprocessing`.

 Соответсвенно, прогнозы будут показываться числом -- 0 или 1. 0 подсвечивается красным, 1 -- зеленым.

Для машинного обучения была выбран ансамбль деревьев(`RandomForest`) быстрый и легкий алгоритм машинного обучения. Качество оценивалось на тестовой выборке, то есть весь датасет делился на обучающую и тестовую.

Отзывы на русском языке, поэтому, понятно, с английским ничего работать не будет. Длина отзыва не так важна, это может повлиять разве что на скорость обработки. Для векторизации использовался обычный bag-of-words.

In [ ]:
import numpy as np
import pandas as pd
import re

from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

In [41]:
file_name = 'review_data.csv'

In [42]:
df = pd.read_csv(file_name)
df.head()

,mark,text,url
0,1.0,"Ставлю оценку отлично, так как банк является с...",https://www.banki.ru/services/responses/bank/r...
1,1.0,Пару месяцев назад мне понадобилось 140-150 т....,https://www.banki.ru/services/responses/bank/r...
2,0.0,Разочарованы работой банка Тинькофф Бизнес.,https://www.banki.ru/services/responses/bank/r...
3,0.0,В одностороннем порядке повысили лимит кредитк...,https://www.banki.ru/services/responses/bank/r...
4,0.0,Дорогой Ситибанк!,https://www.banki.ru/services/responses/bank/r...


In [53]:
model = make_pipeline(
    CountVectorizer(),
    RandomForestClassifier()
)

In [56]:
index_list = list(df.index)
np.random.shuffle(index_list)

cnt_learn = int(2*len(index_list)/3)
cnt_test = len(index_list) - cnt_learn

index_learn = index_list[:cnt_learn]
index_test = index_list[cnt_learn:]

learn_dataset = df.loc[index_learn, 'text']
test_dataset = df.loc[index_test, 'text']

learn_y = df.loc[index_learn, 'mark']
test_y = df.loc[index_test, 'mark']

In [58]:
model.fit(learn_dataset, learn_y)

Pipeline(memory=None,
     steps=[('countvectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
  ...n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))])

In [62]:
((model.predict(test_dataset) == test_y).sum())/(test_y.shape[0])

0.79041095890410962

In [67]:
import pickle
with open('model.md', 'wb') as file_to:
    pickle.dump(model, file_to)

<img src="files/mark_0.png">

<img src="files/mark_1.png">